In [1]:
import pandas as pd
import pandas.io.sql as pdsql
from pandas import DataFrame
import numpy as np
import scipy as sp
from numpy import NaN, Inf, arange, isscalar, asarray, array

import mysql.connector

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib import dates
import matplotlib.font_manager as font_manager
import seaborn as sns

#맑은고딕체
sns.set(style="whitegrid", font="Malgun Gothic", font_scale=1.0)
matplotlib.rcParams['figure.figsize'] = [12, 8]
fp = font_manager.FontProperties(fname="C:\\WINDOWS\\Fonts\\malgun.TTF", size=10)

In [2]:
def comma_volume(x, pos):  # formatter function takes tick label and tick position
    s = '{:0,d}K'.format(int(x/1000))
    return s

def comma_price(x, pos):  # formatter function takes tick label and tick position
    s = '{:0,d}'.format(int(x))
    return s

def comma_percent(x, pos):  # formatter function takes tick label and tick position
    s = '{:+.2f}'.format(x)
    return s

major_date_formatter = dates.DateFormatter('%Y-%m')
minor_date_formatter = dates.DateFormatter('%m')
price_formatter = ticker.FuncFormatter(comma_price)
volume_formatter = ticker.FuncFormatter(comma_volume)
percent_formatter = ticker.FuncFormatter(comma_percent)

In [3]:
class NumpyMySQLConverter(mysql.connector.conversion.MySQLConverter):
    """ A mysql.connector Converter that handles Numpy types """

    def _float32_to_mysql(self, value):
        return float(value)

    def _float64_to_mysql(self, value):
        return float(value)

    def _int32_to_mysql(self, value):
        return int(value)

    def _int64_to_mysql(self, value):
        return int(value)

    def _timestamp_to_mysql(self, value):
        return value.to_datetime()

데이타베이스_설정값 = {
    'host': '127.0.0.1',
    'user': 'root',
    'password': 'PASSWORD',
    'database': 'DB_NAME',
    'raise_on_warnings': True,
}

MySQL_POOL_SIZE = 2

def mysqlconn():
    conn = mysql.connector.connect(pool_name="pool_name", pool_size=MySQL_POOL_SIZE, **데이타베이스_설정값)
    conn.set_converter_class(NumpyMySQLConverter)
    return conn

In [4]:
# 데이타를 기간에 맞게 잘라내는 함수
def 기간(dataframe, 시작기간=None, 종료기간=None):
    df = dataframe.copy()

    if (시작기간 is None) and (종료기간 is None):
        pass
    elif (시작기간 is None) and not(종료기간 is None):
        df = df[:종료기간]
    elif not(시작기간 is None) and (종료기간 is None):
        df = df[시작기간:]
    elif not(시작기간 is None) and not(종료기간 is None):
        df = df[시작기간:종료기간]

    return df

def 배당수익률(수익률=6):
    result = DataFrame()

    query = """
    SELECT A.날짜, A.종목코드, B.종목명, A.배당수익률, B.전일종가, CAST(((B.주식수 * B.전일종가) / 100000000) AS UNSIGNED) AS 시가총액, A.영업이익, A.유보율, A.ROE  
    FROM 재무정보 A, 종목코드 B  
    WHERE 배당수익률 >= %s AND 날짜 > '2019-01-01' AND A.종목코드=B.종목코드 AND ROE >= 10
        AND ((B.시장구분 IN ('KOSPI' , 'KOSDAQ'))
            AND (NOT ((B.종목명 LIKE '%%스팩')))
            AND (NOT ((B.종목명 LIKE '%%SPAC')))
            AND (NOT ((B.종목상태 LIKE '%%관리종목%%')))
            AND (NOT ((B.종목상태 LIKE '%%거래정지%%')))
            AND (NOT ((B.감리구분 LIKE '%%투자경고%%')))
            AND (NOT ((B.감리구분 LIKE '%%투자주의%%')))
            AND (NOT ((B.감리구분 LIKE '%%환기종목%%')))
            AND (NOT ((B.종목명 LIKE '%%ETN%%')))
            AND (NOT ((B.종목명 LIKE '%%0호')))
            AND (NOT ((B.종목명 LIKE '%%1호')))
            AND (NOT ((B.종목명 LIKE '%%2호')))
            AND (NOT ((B.종목명 LIKE '%%3호')))
            AND (NOT ((B.종목명 LIKE '%%4호')))
            AND (NOT ((B.종목명 LIKE '%%5호')))
            AND (NOT ((B.종목명 LIKE '%%6호')))
            AND (NOT ((B.종목명 LIKE '%%7호')))
            AND (NOT ((B.종목명 LIKE '%%8호')))
            AND (NOT ((B.종목명 LIKE '%%9호')))
            AND (NOT ((B.종목명 LIKE '%%0')))
            AND (NOT ((B.종목명 LIKE '%%1')))
            AND (NOT ((B.종목명 LIKE '%%2')))
            AND (NOT ((B.종목명 LIKE '%%3')))
            AND (NOT ((B.종목명 LIKE '%%4')))
            AND (NOT ((B.종목명 LIKE '%%5')))
            AND (NOT ((B.종목명 LIKE '%%6')))
            AND (NOT ((B.종목명 LIKE '%%7')))
            AND (NOT ((B.종목명 LIKE '%%8')))
            AND (NOT ((B.종목명 LIKE '%%9')))
            AND (NOT ((B.종목상태 LIKE '%%증거금100%%')))) 
    ORDER BY 배당수익률 DESC
    """ % (수익률)

    conn = mysqlconn()
    df = pdsql.read_sql_query(query, con=conn)
    conn.close()
    
    df = df.drop_duplicates(['날짜','종목코드'], keep='last')
    df['적정주가비율'] = df['시가총액'] / (df['영업이익'] * df['ROE'] * 10)
    df['목표주가'] = df['배당수익률'] * df['전일종가'] / 3
    df['목표주가'] = df['목표주가'].astype(int)
    
    return df 

In [5]:
# 배당수익률이 5% 이상인 종목 선정
df = 배당수익률(5)
print(len(df))
df.tail()

28


,날짜,종목코드,종목명,배당수익률,전일종가,시가총액,영업이익,유보율,ROE,적정주가비율,목표주가
32,2019-12-31,033660,아주캐피탈,5.22,12000,6906,299.0,172.82,11.72,0.197073,20880
34,2019-12-31,046110,한일네트웍스,5.19,4650,556,28.0,1028.75,15.26,0.130125,8044
35,2019-12-31,038390,레드캡투어,5.11,15350,1318,346.0,3876.53,13.77,0.027663,26146
37,2019-12-31,049720,고려신용정보,5.06,5600,801,29.0,221.82,38.96,0.070895,9445
38,2019-12-31,003780,진양산업,5.01,4035,525,43.0,447.40,11.73,0.104086,6738
